# Web & Scraping

Requires: urllib, requests, socket, re, lxml, io, bs4
План:
* 101 101
* HTML 101
* HTTP + REST 101
* Web scraping

In [ ]:
# Убедитесь, что вотэтовотвсё установлено
import bs4
import io
import lxml
import re
import requests
import socket
import urllib

Заметка на полях:
# 101 101

В американской системе образования курсы имеют индекс, например, [CS101](https://lagunita.stanford.edu/courses/Engineering/CS101/Summer2014/about).

101 -- это вводный курс в какую-то тему, просто так повелось. Intro.


# Интернет 101

Что происходит, когда вы вбиваете в поисковике адрес и жмете Enter?

URL:  
`<схема>:[//[<логин>[:<пароль>]@]<хост>[:<порт>]][/<URL‐путь>][?<параметры>][#<якорь>]`  
Например:  
`https://pp.userapi.com/c834101/v834101778/13450d/9yxFBjsPxN8.jpg`

Вы выступаете **клиентом**

1. **DNS**: URL -> IP-адрес и порт **сервера** (87.240.129.71:443)
2. **HTTP(S)**: IP + URL-путь + параметры -> **ресурс** (текст (в т.ч. *HTML-документ*), картинка, звук и т.д.)
3. **Движок браузера** пытается отрисовать документ
4. *Скорее всего*, внутри документа написано, что ему требуются еще какие-то ресурсы, у них есть URI, т.е. возвращаемся к п.1
5. Пока все это происходит, (и после того, как произошло), выполняется **JavaScript**-код

# HTML 101

HyperText Markup Language

HTML -- язык разметки, который используется для web-страниц.

Это механизм для получения структурного текста, который понимают браузеры.

Структура в тексте задаётся вложенными тегами, теги определяют то, как текст будет показан (отрендерен).

Это тег: `<тег>`, теги бывают открывающими (`<тег>`) и закрывающими (`</тег>`).

Пример HTML-разметки:

```html
<!DOCTYPE html>
<html>
   <head>
      <meta charset="utf-8" />
      <title>HTML Document</title>
   </head>
   <body>
      <p> <!-- p -- это параграф, а в такие странные скобки заключается комментарий -->
         <b>
            Этот текст будет полужирным, <i>а этот — ещё и курсивным</i>.
         </b>
      </p>
   </body>
</html>
```

Так он выглядит после рендеринга:

<!DOCTYPE html>
<html>
   <head>
      <meta charset="utf-8" />
      <title>HTML Document</title>
   </head>
   <body>
      <p> <!-- p -- это параграф, а в такие странные скобки заключается комментарий -->
         <b>
            Этот текст будет полужирным, <i>а этот — ещё и курсивным</i>.
         </b>
      </p>
   </body>
</html>

Видно, что HTML-разметка имеет древовидную структуру:<br>
каждый тег (вершина дерева) имеет 0 (тогда это лист дерева) или больше (тогда это внутренняя вершина) вложенных в него тегов.

Значит, чтобы доставать из HTML какую-то информацию, можно использовать его структуру.

Мы этим займёмся чуть позже.

### P.S. 
На самом деле, такую же структуру имеет и формат XML (другой язык разметки).

Формально, HTML -- это более стандартизированное подмножество XML.

### P.P.S.

А ещё у тегов бывают **атрибуты**:

```html
<a href="http://example.com">Ссылка на example.com</a>
```
Отрендерим этот кусок:
<a href="http://example.com">Ссылка на example.com</a>

### P.P.P.S.

А этот текст (безусловно, имеющий структуру), был написан с помощью другого языка разметки -- **Markdown**.

## HTTP 101

HyperText Transfer Protocol

Протокол передачи гипертекста:
* **Протокол** -- набор логических соглашений о взаимодействии программ, стандарт;
* **передача** -- мы живём в интернете, т.е. мир не ограничивается нашим компьютером => информацию надо передавать;
* **гипертекст** -- HTML, например.

Основная идея: все компьютеры делятся на **сервера** (те, что хранят информацию) и **клиенты** (те, что запрашивают её).

На самом деле, в HTTP мы передаём не только гипертекст, а **ресурсы** -- философское обобщение, куда попадает и гипертекст, и картинки, и музыка и т.д. Но стандарт подходит для всего этого.

Каждый ресурс имеет адрес -- **URI** (Uniform Resource Identifier) -- аналог пути в системе компьютера.

Не будем говорить про низкоуровневую сторону вопроса: стек TCP/IP, сокеты, порты и т.д.

## Структура HTTP

Чтобы получить данные, клиент делает запрос на сервер. В запросе должны быть 3 части:
* Starting line -- определяет тип сообщения;
* Headers -- характеризуют тело сообщения, параметры передачи и прочие сведения;
* Message body -- непосредственно данные сообщения.

Ответное сообщение от сервера имеет такую же структуру.

Чтобы посылать эти запросы "руками", а не через браузер, можно использовать утилиту `curl`. Ей и воспользуемся.

In [ ]:
!curl example.com/index.html -v

## Методы HTTP

HTTP метод определяет операцию, которую мы хотим произвести над ресурсом.

Самые частые методы: GET, POST, PUT, DELETE (но есть и другие).

* **GET** -- запрос содержимого ресурса. <br>GET-запросы идемпотентны. Поэтому они могут кэшироваться.
* **POST** -- передача данных на ресурс (например, при отправке комментария на форуме или вводе пароля на сайте). <br> Не идемпотентны => при отправке одного и того же комментария на форум он появится там дважды.  <br>Не кэшируются.
* **PUT** -- передача данных в конкретный URI (изменение существующего ресурса). Не кэшируются.
* **DELETE** -- удаление ресурса.

## Коды HTTP

В ответном сообщении придёт код ответа HTTP, который определяет результат выполнения операции.

Самые частые коды: `200 OK`, `400 BadRequest`, `404 Not Found`, `500 Internal Server Error`.

Общий обзор кодов:
* **1xx** -- Informational. Информационные коды, например, `102 Processing` (запрос пока обрабатывается);
* **2xx** -- Success. Успех. Всё хорошо, запрос отработал и ничего не сломал. По крайней мере, пока.
* **3xx** -- Redirection. Перенаправление на другой ресурс/страницу.
* **4xx** -- Client error. Ошибка клиента (неверные данные запроса или неправильный путь).
* **5xx** -- Server error. Что-то сломалось на сервере (там поделили на ноль, например).

<a href="https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BA%D0%BE%D0%B4%D0%BE%D0%B2_%D1%81%D0%BE%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D1%8F_HTTP">Полный список кодов.</a>

## Разбор примера

Давайте ещё раз посмотрим на запрос к example.com/index.html.

Служебная информация от `curl`:

```
*   Trying 2606:2800:220:1:248:1893:25c8:1946...
*   Trying 93.184.216.34...
* Connected to example.com (2606:2800:220:1:248:1893:25c8:1946) port 80 (#0)
```
Запрос клиента:
```
> GET /index.html HTTP/1.1    # Starting line: метод GET, URI -- /index.html, версия протокола -- HTTP/1.1
> Host: example.com           # Заголовки сообщения
> User-Agent: curl/7.47.0
> Accept: */*
>                             # Пустое тело, т.к. мы ничего не передали на сервер.
```

Ответ сервера:
```
< HTTP/1.1 200 OK                          # Starting line: версия протокола и код ответа
< Cache-Control: max-age=604800            # Заголовки ответа
< Content-Type: text/html; charset=UTF-8
< Date: Tue, 19 Mar 2019 21:25:21 GMT
< Etag: "1541025663+gzip+ident"
< Expires: Tue, 26 Mar 2019 21:25:21 GMT
< Last-Modified: Fri, 09 Aug 2013 23:54:35 GMT
< Server: ECS (bsa/EB1E)
< Vary: Accept-Encoding
< X-Cache: HIT
< Content-Length: 1270
<                                          # Пустая строка -- нужна по стандарту
<!doctype html>                            # Тело ответа -- HTML-документ
<html>
<head>
    <title>Example Domain</title>
. . .
```


## Инструменты для работы с HTTP

Больше <a href="https://github.com/MVCionOld/mipt-atp-caos-seminars/tree/master/seminar-24">примеров и информации</a> можно найти здесь.

## REST

 <a href="https://en.wikipedia.org/wiki/Representational_state_transfer">Representational State Transfer</a> 

Набор дополнительных ограничений на HTTP, указывающий, как правильно работать с методами, кодами и т.д.

Можно назвать его аналогом pep для Питона.
  
Rest архитектура должна удовлетворять следующим критериям:
 1. **Client-Server.** Система должна быть разделена на клиентов и на серверов. Разделение интерфейсов означает, что, например, клиенты не связаны с хранением данных, которое остается внутри каждого сервера, так что мобильность кода клиента улучшается. Серверы не связаны с интерфейсом пользователя или состоянием, так что серверы могут быть проще и масштабируемы. Серверы и клиенты могут быть заменяемы и разрабатываться независимо, пока интерфейс не изменяется.
 2. **Stateless.** Сервер не должен хранить какой-либо информации о клиентах. В запросе должна храниться вся необходимая информация для обработки запроса и если необходимо, идентификации клиента.
 3. **Cache․** Каждый ответ должен быть отмечен является ли он кэшируемым или нет, для предотвращения повторного использования клиентами устаревших или некорректных данных в ответ на дальнейшие запросы.
 4. **Uniform Interface.** Единый интерфейс определяет интерфейс между клиентами и серверами. Это упрощает и отделяет архитектуру, которая позволяет каждой части развиваться самостоятельно.
  
**HTTP методы для создания RESTful сервисов**
 1. **HTTP метод GET** используется для получения (или чтения) представления ресурса. В случае “удачного” (или не содержащего ошибок) адреса, GET возвращается представление ресурса в формате XML или JSON в сочетании с кодом состояния `HTTP 200 (OK)`. В случае наличия ошибок обычно возвращается код `HTTP 404 (NOT FOUND)` или `HTTP 400 (BAD REQUEST)`.
 2. **HTTP метод POST** запрос наиболее часто используется для создания новых ресурсов. При успешном создании ресурса возвращается `HTTP 201 (CREATED)`.
 3. **HTTP метод PUT** обычно используется для предоставления возможности обновления ресурса. Тело запроса при отправке PUT-запроса к существующему ресурсу URI должно содержать обновленные данные оригинального ресурса (полностью, или только обновляемую часть). При успешном обновлении посредством выполнения PUT запроса возвращается код `HTTP 200 (OK)` (или `HTTP 204 (NO CONTENT)` если не был передан какой либо контент в теле ответа).
 4. **HTTP метод DELETE** используется для удаления ресурса, идентифицированного конкретным URI (ID). При успешном удалении возвращается `HTTP 200 (OK)`, совместно с телом ответа, содержащим данные удаленного ресурса. Также возможно использование `HTTP 204 (NO CONTENT)` без тела ответа.
 
Иногда на практике ограничиваются только методами GET (чтение) и POST (создание, обновление и удаление). А бывают случаи, когда используют только POST:)

# Web scraping. Практика!

* Получение html
* Парсинг html

## Получение html

Варианты инструментов:

* urllib 
* requests (de-facto standard)
* socket (low-level)



### urllib 

In [ ]:
import urllib.request

In [ ]:
response = urllib.request.urlopen('http://example.com/')
html = response.read()
print(html)

In [ ]:
html = html.decode('utf-8')
print(html)

На всякий случай сохраним этот html, пригодится.

In [ ]:
with open('example.com.txt', 'w', encoding='utf-8') as f:
    f.write(html)

А что, кроме html?

In [ ]:
print(dir(response))

Всё, что есть в HTTP-сообщении.

In [ ]:
print(response.url)
print(response.msg)
print(response.code)
print('Заголовки: \n{}'.format(response.headers))
# . . .

### requests
HTTP for Humans

In [ ]:
import requests

In [ ]:
response = requests.get('http://example.com')
response

In [ ]:
print(dir(response))

In [ ]:
print(response.url)
print(response.connection)
print(response.headers)
print(response.ok)
print(response.status_code)
print(response.encoding)
print(response.links)

In [ ]:
print(response.text)

## Парсинг html

Варианты инструментов:
* re
* lxml
* BeautifulSoup

In [ ]:
with open('example.com.txt', 'r', encoding='utf-8') as f:
    html = f.read()
print(html)

### lxml

Воспользуемся библиотекой, которая знает про структуру XML (и HTML).

In [ ]:
from lxml import etree
from io import StringIO, BytesIO

In [ ]:
parser = etree.HTMLParser()
tree = etree.parse(StringIO(html), parser)
tree

In [ ]:
print(dir(tree))

In [ ]:
print(tree.getroot())
print(etree.tostring(tree.getroot(), pretty_print=True, method='html'))

In [ ]:
paragraphs = tree.xpath('//p')
for p in paragraphs:
    print(p.text)  

In [ ]:
for p in paragraphs:
    print(etree.tostring(p, pretty_print=True, method='html'))  

In [ ]:
hrefs = tree.xpath('//a')
for href in hrefs:
    print(href.text)  
    print(href.attrib)  

In [ ]:
specific_hrefs = tree.xpath('//a[@href="https://www.iana.org/domains/example"]')
specific_hrefs

### BeautifulSoup

Ещё один вариант, который хорошо подходит для парсинга HTML.

In [ ]:
# try:
from bs4 import BeautifulSoup
# except:
#     !pip install bs4 

In [ ]:
soup = BeautifulSoup(html, 'lxml')

In [ ]:
paragraphs = soup.find_all('p')
paragraphs

In [ ]:
hrefs = soup.find_all('a')
hrefs

In [ ]:
hrefs = soup.find_all('a', href='https://www.iana.org/domains/example')
hrefs

In [ ]:
hrefs = soup.find_all('a', href='https://www.other-website.org/domains/example')
hrefs

In [ ]:
dir(soup)

## Performance

In [ ]:
%timeit re.findall(r'<p>([\w\W]*?)</p>', html)
%timeit tree.xpath('//p')
%timeit soup.find_all('p')

Мораль: regex -- это быстро.

BeautifulSoup чуть дольше lxml, т.к. он для удобства работы преобразует документ в некий внутренний формат -- собственно, суп.